In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import MinMaxScaler


#### Importing Dataset from Yahoo Finance

**Common Tickers include: ['FB','AMZN','GOOG','TWTR','TSLA']**

In [ ]:
import yfinance as yf
stock_symbol = 'TWTR'
data = yf.download(tickers=stock_symbol,period='2y',interval='1d')
data = data.reset_index()
data.tail()


#### Visualizing the Real Stock Trend

In [ ]:
plt.figure(figsize=(14,5))
plt.plot(data['Open'],color='black',label='true trend')
plt.title('%s Stock Prices'%stock_symbol)
plt.xlabel('Time')
plt.ylabel('%s Stock Prices'%stock_symbol)
plt.legend()
plt.show()


#### Marking a training timeframe of 60 days

In [ ]:
test_size = 60
data_train = data[:-test_size]
data_test = data[-test_size:]
data.shape,data_train.shape,data_test.shape


#### Preparing the training data

In [ ]:
training_data = data_train.drop(['Date','Adj Close'],axis=1)
training_data.head()


In [ ]:
scaler = MinMaxScaler()
training_data = scaler.fit_transform(training_data)
training_data[0]


In [ ]:
# Preparing the training set
x_train = []
y_train = []

for i in range(test_size, training_data.shape[0]):
    x_train.append(training_data[i-test_size:i])
    y_train.append(training_data[i,0])
# last 60 days will be the training set - X
# 61st day is the target value - Y

x_train = np.array(x_train)
y_train = np.array(y_train)

x_train.shape,y_train.shape
# gives rows,columns,dimentional data


#### Preparaing testing data

In [ ]:
past_test_days = data_train.tail(test_size)
df = pd.concat([past_test_days,data_test])
df = df.drop(['Date','Adj Close'],axis='columns')
df.shape, df.head()


In [ ]:
df = scaler.transform(df)
df[0]


In [ ]:
x_test = []
y_test = []

for i in range(test_size,df.shape[0]):
    x_test.append(df[i-test_size:i])
    y_test.append(df[i,0])

x_test, y_test = np.array(x_test),np.array(y_test)
x_test.shape, y_test.shape


#### Defining the Scaling Label

In [ ]:
# below is the scaling label, 
# therefore we need to divide our y_pred with the below scaling value it used to scale the data
scaler.scale_


In [ ]:
scale = 1/8.50296711e-04
scale


In [ ]:
y_test = y_test*scale


### Bidirectional GRU - (Gated Recurrent Unit)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, GRU, Dropout, Bidirectional


In [ ]:
x_train.shape[1],x_train.shape[2]


In [ ]:
model = Sequential()

# 1st layer
model.add(Bidirectional(GRU(units=40,activation='relu',return_sequences=True),input_shape=(x_train.shape[1],x_train.shape[2])))
model.add(Dropout(0.2))

# 2nd layer
model.add(Bidirectional(GRU(units=60,activation='relu',return_sequences=True)))
model.add(Dropout(0.2))

# 3rd layer
model.add(Bidirectional(GRU(units=80,activation='relu',return_sequences=True)))
model.add(Dropout(0.2))

# Final layer
model.add(Bidirectional(GRU(units=120,activation='relu')))
model.add(Dropout(0.5))

model.add(Dense(units=1))

model.summary()


In [ ]:
model.compile(optimizer='adam',loss='mean_squared_error',metrics='accuracy')


In [ ]:
def forecast():
    model.fit(x_train,y_train,epochs=300,batch_size=32)
    y_pred = model.predict(x_test)
    y_pred = y_pred*scale
    return y_pred


In [ ]:
def first_forecast():
    history = model.fit(x_train,y_train,epochs=300,batch_size=32)
    y_pred = model.predict(x_test)
    y_pred = y_pred*scale
    return history,y_pred


In [ ]:
result_forecast = []
simulation = 1
print("Simulation 1")
history,first_result = first_forecast()
result_forecast.append(first_result)
for i in range(1,simulation):
    print("Simulation %d"%(i+1))
    result_forecast.append(forecast())


#### Calculating the Accuracy using RMSE

In [ ]:
# MAPE - Mean Absolute Percentage Error
def calculate_accuracy(real, predict):
    real = np.array(real) + 1
    predict = np.array(predict) + 1
    # mean Absolute Percentage Error - MAPE
    percentage = 1 - np.mean(np.abs((real-predict)/real))
    return percentage * 100
mean_accuracy = []
for i in range(simulation):
    mean_accuracy.append(calculate_accuracy(y_test, result_forecast[i]))
    
mean_accuracy


In [ ]:
# RMSE - Root Mean Squared Error 
temp = 1 - np.sqrt(np.mean(np.square((y_test - result_forecast[0])/y_test)))
temp*100


#### Visualization

In [ ]:
plt.figure(figsize=(14,5))
for i in range(simulation):
    plt.plot(result_forecast[i],label='forecast %d'%(i+1))
plt.plot(y_test,color='black',label='true trend')
plt.title('Mean Accuracy: %.4f'%np.mean(mean_accuracy))
plt.xlabel('Time')
plt.ylabel('%s Stock Prices'%stock_symbol)
plt.legend()
plt.show()


In [ ]:
history.history.keys()

In [ ]:
plt.plot(history.history['accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.title('Bidirectional GRU Model, Accuracy vs Epoch')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.title('Bidirectional GRU Model, Loss vs Epoch')
plt.show()